# bioimageio.core usage examples

In [1]:
import os
import hashlib

import bioimageio.core
import imageio
import napari
import numpy as np
import xarray as xr

from bioimageio.core.prediction_pipeline import create_prediction_pipeline

In [2]:
def show_images(*images, names=None):
    v = napari.Viewer()
    for i, im  in enumerate(images):
        name = None if names is None else names[i]
        v.add_image(im, name=name)

## Loading a model
https://bioimage.io/#/?id=10.5072%2Fzenodo.881940

In [3]:
# rdf_url = "https://sandbox.zenodo.org/record/881989/files/rdf.yaml"
rdf_url = "https://sandbox.zenodo.org/record/881989/files/rdf.yaml?download=1"
rdf_doi = "10.5072/zenodo.881940"
rdf_path = "/home/pape/Downloads/dsb-nuclei-boundarymodelnew_pytorch_state_dict.zip"

In [4]:
# load model from link to rdf.yaml
# TODO does not work yet
# model_resource = bioimageio.core.load_resource_description(rdf_url)

In [5]:
# TODO does not work yet
# model_resource = bioimageio.core.load_resource_description(rdf_doi)

In [6]:
# load model from path to zip
# TODO explain that this also works with unpacked model and path to rdf.yaml
model_resource = bioimageio.core.load_resource_description(rdf_path)

## Prediction with the model

In [8]:
input_image = np.load(model_resource.test_inputs[0])

In [9]:
# prediction with numpy arrays
# explain!
def predict_numpy(model, input_, devices=None, weight_format=None):
    pred_pipeline = create_prediction_pipeline(
        bioimageio_model=model, devices=devices, weight_format=weight_format
    )

    # explain!
    axes = tuple(model.outputs[0].axes)
    input_tensor = xr.DataArray(input_, dims=axes)
    prediction = pred_pipeline(input_tensor)[0]
    return prediction

In [10]:
prediction = predict_numpy(model_resource, input_image)
show_images(input_image, prediction, names=["image", "prediction"])

/home/pape/Work/software/conda/miniconda3/envs/bio-core-dev/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /tmp/pip-req-build-xi8rb33b/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [11]:
# prediction with image from disc
from bioimageio.core.prediction import predict_image
outputs = ["prediction.tif"]
predict_image(
    model_resource, model_resource.test_inputs, outputs
)
fg_pred = imageio.imread("prediction-c0.tif")
bd_pred = imageio.imread("prediction-c1.tif")
show_images(input_image, fg_pred, bd_pred,
            names=["image", "foreground-prediction", "boundary-prediction"])

In [12]:
# prediction with image from disc
# TODO download dsb data, and run multi-image prediction with tiling and padding
from bioimageio.core.prediction import predict_images

## Create a biomiage.io model package

In [13]:
# create a new model with different post-processing
from bioimageio.core.build_spec import build_model

model_root = "./new_model"
os.makedirs(model_root, exist_ok=True)

threshold = 0.5
new_output = prediction > threshold
new_output_path = f"{model_root}/new_test_output.npy"
np.save(new_output_path, new_output)

# explain!
preprocessing = [
    {prep.name: prep.kwargs for prep in model_resource.inputs[0].preprocessing}
]
postprocessing = [{"binarize": {"threshold": threshold}}]

model_path = os.path.join(model_resource.root_path, "unet.py")
assert os.path.exists(model_path), model_path
model_source = f"{model_path}:UNet2d"

cite = {cite_entry.text: cite_entry.url for cite_entry in model_resource.cite}
name = "new-model1"

weight_file = model_resource.weights["pytorch_state_dict"].source
with open(weight_file, "rb") as f:
    weight_hash = hashlib.sha256(f.read()).hexdigest()
parent = (rdf_doi, weight_hash)

zip_path = os.path.join(model_root, f"{name}.zip")
new_model_raw = build_model(
    weight_file,
    test_inputs=model_resource.test_inputs,
    test_outputs=[new_output_path],
    output_path=zip_path,
    name=name,
    description="nucleus segmentation model with thresholding",
    authors=[{"name": "Jane Doe"}],
    license="CC-BY-4.0",
    documentation=model_resource.documentation,
    covers=[str(cover) for cover in model_resource.covers],
    tags=["nucleus-segmentation"],
    cite=cite,
    parent=parent,
    root=model_root,
    source=model_source,
    model_kwargs=model_resource.kwargs,
    preprocessing=preprocessing,
    postprocessing=postprocessing
)

invalid local relative path: documentation.md


In [14]:
new_model = bioimageio.core.load_resource_description(zip_path)
prediction = predict_numpy(new_model, input_image)
show_images(input_image, prediction, names=["input", "binarized-prediction"])

## Add different weight format and package model with new weights

In [16]:
from bioimageio.core.weight_converter.torch import convert_weights_to_pytorch_script
from bioimageio.core.build_spec import add_weights

weight_path = os.path.join(model_root, "weights.torchscript")
convert_weights_to_pytorch_script(new_model, weight_path)
zip_path = f"{model_root}/new_model2.zip"
new_model2_raw = add_weights(new_model_raw, weight_path, weight_type="pytorch_script", output_path=zip_path)

invalid local relative path: ./documentation.md


In [17]:
new_model = bioimageio.core.load_resource_description(zip_path)
prediction = predict_numpy(new_model, input_image, weight_format="pytorch_script")
show_images(input_image, prediction, names=["input", "binarized-prediction"])

In [18]:
# saving model packages
bioimageio.core.export_resource_package(new_model2_raw, output_path="another_model.zip")

invalid local relative path: ./documentation.md


'another_model.zip'